In [2]:
import yaml
import os
import zarr
import fibsem_tools as fst
import numpy as np

In [4]:
with open("../all_data.yaml") as f:
    datas = yaml.safe_load(f)

In [ ]:
for key, crops in datas.items():
    print(key, "-", os.path.exists(f"/nrs/cellmap/data/{key}"), "-", os.path.exists(f"/nrs/cellmap/data/{key}/staging/groundtruth.zarr"))
    classes = set()
    for crop in crops:
        if not os.path.exists(f"/nrs/cellmap/data/{key}/staging/groundtruth.zarr/{crop}"):
            print(crop)
        else:
            crop_zarr = zarr.open(f"/nrs/cellmap/data/{key}/staging/groundtruth.zarr/{crop}", "r")
            classes = classes.union(set(crop_zarr.attrs["cellmap"]["annotation"]["class_names"]))
    print(classes)
            

In [19]:
with open("../data.yaml") as f:
    datas = yaml.safe_load(f)

In [6]:
root_gt_path = "/nrs/saalfeld/heinrichl/data/cellmap_labels/fly_organelles/"
root_data_path = "/nrs/cellmap/data/"

In [ ]:
for key, ds_info in datas.items():
    print(key, "-", os.path.exists(f"{root_gt_path}{key}"), "-", os.path.exists(f"{root_gt_path}{key}/groundtruth.zarr"), "-", os.path.exists(f"{ds_info['raw']}/s0"))
    for crop in ds_info["crops"]:
        if not os.path.exists(f"{root_gt_path}{key}/groundtruth.zarr/{crop}"):
            print(crop)

In [5]:
def fix_offset(data, crop, orig_offset, new_offset):
    diff_offset = np.array(orig_offset) - np.array(new_offset)
    crop_root = fst.access(f"{root_gt_path}{data}/groundtruth.zarr/{crop}", mode="r+")
    for lbl in list(crop_root.keys()):
        crop_label_root = crop_root[lbl]
        ms_attrs = crop_label_root.attrs["multiscales"]
        for ds in range(len(ms_attrs[0]["datasets"])):
            for k in range(len(ms_attrs[0]["datasets"][ds]["coordinateTransformations"])):
                if ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["type"] == "translation":
                    transl = ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["translation"] 
                    ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["translation"] = list(np.array(transl) - diff_offset)
        crop_label_root.attrs["multiscales"] = ms_attrs

In [ ]:
# def fix_scale(data, crop):
#     crop_root = fst.access(f"{root_gt_path}{data}/groundtruth.zarr/{crop}", mode="r+")
#     for lbl in list(crop_root.keys()):
#         crop_label_root = crop_root[lbl]
#         ms_attrs = crop_label_rrot.attrs["multiscales"]
#         for ds in range(len(ms_attrs[0]["datasets"])):
#             for k in range(len(ms_attrs[0]["datasets"][ds]["coordinateTransformations"])):
#                 if ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["type"] == "scale":
#                     scale = ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["scale"]
#                     scale = [scale[1],]*3
#                     print(scale)
#                     #ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["scale"]

In [ ]:
# for crop in datas["culture/jrc_ctl-id8-1"]["crops"]:
#     fix_scale("cutlure/jrc_ctl-id8-1", crop)

In [14]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        lbl = list(crop_root.keys())[0]
        crop_datasets = crop_root[lbl].attrs["multiscales"][0]["datasets"]
        for cds in crop_datasets:
            if cds["path"] == "s0":
                crop_res = cds["coordinateTransformations"][0]["scale"]
                crop_off = cds["coordinateTransformations"][1]["translation"]
                crop_off_arr = np.array(crop_off)
                crop_res_arr = np.array(crop_res)
                corner_off_arr = (crop_off_arr + raw_res_arr/2. - crop_res_arr/2.)
                print("########################")
                if np.all(corner_off_arr % raw_res_arr == 0) and np.all(corner_off_arr % crop_res_arr == 0):
                    print(crop, "GOOD", raw_res, crop_res, crop_off)
                else:
                    print(crop, "BAD")
                    print("RAW -", raw_res, "; CROP -", crop_res, "@", crop_off,"->", corner_off_arr)
                    if np.all(crop_res < raw_res):
                        crop_off_arr_suggested = crop_off_arr - crop_res_arr/2.
                        corner_off_arr_suggested = (crop_off_arr_suggested + raw_res_arr/2. - crop_res_arr/2.)
                        if np.all(corner_off_arr_suggested % raw_res_arr == 0) and np.all(corner_off_arr_suggested % crop_res_arr == 0):
                            print("SUGGESTED WORKS:", "suggested offset:", list(crop_off_arr_suggested))
                            fix_offset(key, crop, crop_off, list(crop_off_arr_suggested))
                        else:
                            print("SUGGESTED STILL BAD")
                    else:
                        crop_off_arr_suggested = crop_off_arr + crop_res_arr/2. - raw_res_arr/2.
                        corner_off_arr_suggested = (crop_off_arr_suggested + raw_res_arr/2. - crop_res_arr/2.)
                        if np.all(corner_off_arr_suggested % raw_res_arr == 0) and np.all(corner_off_arr_suggested % crop_res_arr == 0):
                            print("SUGGESTED WORKS:", "suggested offset:", list(crop_off_arr_suggested))
                            fix_offset(key, crop, crop_off, list(crop_off_arr_suggested))
                        else:
                            print("SUGGESTED STILL BAD")
                        #print("IS DIVISIBLE: ", np.all(crop_off_arr%crop_res_arr == 0))
    print("---------------------------------------------------------------------------------------------------------------------------------------------")

tissue/other/jrc_22ak351-leaf-3m
########################
crop352 GOOD [8.0, 8.0, 8.0] [512.0, 512.0, 512.0] [252.0, 252.0, 252.0]
########################
crop385 GOOD [8.0, 8.0, 8.0] [64.0, 64.0, 64.0] [25628.0, 12828.0, 28.0]
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_cos7-1a
########################
crop234 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [2760.0, 5160.0, 10670.0]
########################
crop236 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [10.0, 5690.0, 11750.0]
########################
crop237 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [2856.0, 4942.0, 10288.0]
########################
crop239 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [3028.0, 4874.0, 9364.0]
########################
crop248 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [90.0, 4300.0, 7922.0]
########################
crop251 GOOD [2.0, 2.0, 2.0] [64.0, 64.0, 64.0] [31.0, 31.0, 31.0]
########################
crop252 GOOD [2.0,

########################
crop254 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [2828.0, 5052.0, 9350.0]
########################
crop256 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [1582.0, 6158.0, 8574.0]
########################
crop257 GOOD [2.0, 2.0, 2.0] [8.0, 8.0, 8.0] [3.0, 2403.0, 803.0]
########################
crop292 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [862.0, 5282.0, 6702.0]
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_cos7-1b
########################
crop235 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [118.0, 6560.0, 15904.0]
########################
crop238 GOOD [2.0, 2.0, 2.0] [8.0, 8.0, 8.0] [3.0, 803.0, 1603.0]
########################
crop240 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [702.0, 5302.0, 11480.0]
########################
crop241 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [1114.0, 6394.0, 15546.0]
########################
crop242 GOOD [2.0, 2.0, 2.0] [2.0, 2.0, 2.0] [1664.0, 6042.0,

In [20]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
        
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        lbl = list(crop_root.keys())[0]
        crop_datasets = crop_root[lbl].attrs["multiscales"][0]["datasets"]
        for cds in crop_datasets:
            if cds["path"] == "s0":
                crop_res = cds["coordinateTransformations"][0]["scale"]
                
                crop_off = cds["coordinateTransformations"][1]["translation"]
                
                crop_off_arr = np.array(crop_off)
                crop_res_arr = np.array(crop_res)
                corner_off_arr = (crop_off_arr + raw_res_arr/2. - crop_res_arr/2.)
                if np.all(crop_res_arr > np.array([8.,8.,8.])):
                    print(f"crop {crop} has {crop_res}")
    print("---------------------------------------------------------------------------------------------------------------------------------------------")
                


culture/jrc_cos7-1a
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_cos7-1b


---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_ctl-id8-1
---------------------------------------------------------------------------------------------------------------------------------------------
tissue/brain/jrc_fly-vnc-1
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_hela-2
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_hela-3
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_jurkat-1
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_macrophage-2
---------------

In [21]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
    offsets = []
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        lbl = list(crop_root.keys())[0]
        crop_datasets = crop_root[lbl].attrs["multiscales"][0]["datasets"]
        for cds in crop_datasets:
            if cds["path"] == "s0":
                crop_res = cds["coordinateTransformations"][0]["scale"]
                
                crop_off = cds["coordinateTransformations"][1]["translation"]
                
                print(f"{crop}:{crop_off}")
                offsets.append(tuple(crop_off))
    if len(set(offsets)) < len(offsets):
        print(f"This has some {len(set(offsets))}, {len(offsets)}")
    print("---------------------------------------------------------------------------------------------------------------------------------------------")
                


culture/jrc_cos7-1a
crop234:[2760.0, 5160.0, 10670.0]
crop236:[10.0, 5690.0, 11750.0]
crop237:[2856.0, 4942.0, 10288.0]
crop239:[3028.0, 4874.0, 9364.0]
crop248:[90.0, 4300.0, 7922.0]
crop252:[240.0, 5000.0, 9600.0]
crop253:[1.0, 4401.0, 8601.0]
crop254:[2828.0, 5052.0, 9350.0]
crop256:[1582.0, 6158.0, 8574.0]
crop257:[3.0, 2403.0, 803.0]
crop292:[862.0, 5282.0, 6702.0]
---------------------------------------------------------------------------------------------------------------------------------------------
culture/jrc_cos7-1b
crop235:[118.0, 6560.0, 15904.0]
crop238:[3.0, 803.0, 1603.0]
crop240:[702.0, 5302.0, 11480.0]
crop241:[1114.0, 6394.0, 15546.0]
crop242:[1664.0, 6042.0, 35962.0]
crop245:[1748.0, 5902.0, 12466.0]
crop249:[1252.0, 6760.0, 34716.0]
crop255:[832.0, 6476.0, 31870.0]
crop258:[1420.0, 5378.0, 10844.0]
crop259:[248.0, 4608.0, 9300.0]
crop291:[702.0, 5302.0, 11480.0]
This has some 10, 11
---------------------------------------------------------------------------------

In [ ]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["coordinateTransformations"]
        if len(datasets) > 0:
            print(f"Check this one: {ds_info['raw']}")
    except KeyError as e:
        print("Different metadata format?")
        continue

In [7]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
        
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        if "all" in crop_root:
            labels, counts = np.unique(crop_root["all"]["s0"], return_counts=True)
            if 0 in labels:
                print(f"{crop} labels: ", labels, "INCLUDES 0", counts)
            else:
                print(f" {crop} labels: ", labels)

tissue/other/jrc_22ak351-leaf-3m
Different metadata format?
tissue/other/jrc_22ak351-leaf-3r
Different metadata format?
culture/jrc_cos7-1a
 crop234 labels:  [ 1  2  3  4  8  9 10 11 12 13 16 17 30 35 36]
 crop236 labels:  [ 3  4  5  8  9 10 11 16 17 30 35 36]
 crop237 labels:  [ 1  2  3  4  6  7  8  9 10 11 16 17 18 19 30 35 36]
 crop239 labels:  [ 1  2  3  4  8  9 10 11 16 17 30 35 36]
 crop243 labels:  [1]
crop247 labels:  [ 0 30 38] INCLUDES 0 [838969232    757158   2148610]
 crop248 labels:  [ 1  2  3  4  5  8  9 10 11 16 17 30 35 36]
crop251 labels:  [0 1] INCLUDES 0 [1079340 3949656]
 crop252 labels:  [ 1  2  3  4  5  8  9 10 11 12 13 16 17 18 19 30 35 36]
crop253 labels:  [ 0 30 36] INCLUDES 0 [866235810    758743    192947]
 crop254 labels:  [ 3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 30 35 36]
 crop256 labels:  [ 3  4  5  8  9 16 17 30 35 36]
crop257 labels:  [ 0 37] INCLUDES 0 [137159008 159840992]
 crop292 labels:  [ 3  4 10 11 16 17 20 21 22 23 24 28 35]
culture/jrc_c

In [ ]:
[ 1  2  8  9 10 11 16 17 35]